In [1]:
import torch
import torch.nn as nn
import torchvision.models as models
import torch.nn.functional as F
import numpy as np

## 依照指示取出模型特定層的資訊

In [2]:
# Define model
class TheModelClass(nn.Module):
    def __init__(self):
        super(TheModelClass, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16* 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Initialize model
model = TheModelClass()


### 取出 self.pool層兩次的輸出，包含：
* x = self.pool(F.relu(self.conv1(x)))
* x = self.pool(F.relu(self.conv2(x)))

In [3]:
outputs= []
def layer1_hook(module, input_, output):
    outputs.append(output)

model.pool.register_forward_hook(layer1_hook)

In [4]:
input_ = torch.randn(1, 3, 32, 32)
output = model(input_)

In [5]:
outputs

[tensor([[[[0.1544, 0.6221, 1.1697,  ..., 0.1739, 0.4679, 0.6816],
           [0.3154, 0.8114, 0.9458,  ..., 0.6272, 0.9503, 0.9678],
           [0.4994, 0.6042, 0.1946,  ..., 0.7775, 0.0000, 0.7290],
           ...,
           [0.3817, 0.0000, 1.1320,  ..., 0.7103, 0.6842, 0.7756],
           [0.5905, 1.2043, 0.5052,  ..., 0.2209, 1.2150, 1.3044],
           [1.0320, 0.5015, 0.9195,  ..., 0.3384, 0.2111, 0.9684]],
 
          [[0.0000, 0.5192, 0.0000,  ..., 0.0000, 1.1406, 0.8040],
           [0.0000, 0.2784, 0.0000,  ..., 0.1608, 0.7624, 0.1421],
           [0.8702, 0.3039, 1.1381,  ..., 0.3141, 0.6758, 0.4186],
           ...,
           [0.4194, 0.1216, 0.0000,  ..., 0.4020, 0.3884, 0.3895],
           [0.4166, 0.4250, 1.1019,  ..., 0.2732, 0.4575, 0.4066],
           [0.6744, 1.1234, 0.9366,  ..., 0.8977, 0.9717, 0.3185]],
 
          [[0.0000, 0.0000, 0.9360,  ..., 0.6206, 1.0438, 0.4273],
           [0.5407, 1.5185, 1.0968,  ..., 0.8979, 0.2655, 0.3465],
           [0.2724, 0.06

In [6]:
print(outputs[0].shape)
print(outputs[1].shape)

torch.Size([1, 6, 14, 14])
torch.Size([1, 16, 5, 5])


## 加入自定義 initialization fuction

#### 對所有Conv2D層使用自定義initialization function
* weight : nn.init.kaiming_normal_
* bias : 全部輸入1

In [7]:
from torch.nn import init

def weights_init(m):
    #classname = m.__class__.__name__
    if isinstance(m, nn.Conv2d):
        #torch.nn.init.xavier_uniform_(m.weight)
        nn.init.kaiming_normal_(m.weight.data,
                                    a=0,
                                    mode='fan_out',
                                    nonlinearity='relu')
        if m.bias is not None:
          m.bias.data.fill_(1)

model.apply(weights_init)

TheModelClass(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

#### 查看 conv層的bias是否皆為1

In [8]:
for name, parameters in model.named_parameters():
    if ('conv' in name) and ('bias' in name):
        print(name, parameters)
        print('\n')

conv1.bias Parameter containing:
tensor([1., 1., 1., 1., 1., 1.], requires_grad=True)


conv2.bias Parameter containing:
tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       requires_grad=True)


